In [ ]:
    def coherence2D(self, interaction_times=None, diagram=None, t_range=None, r=10, parallel=False):
        '''
        computes the 2D coherence plot for a single 3D order diagram with fixed population time.
        It can parallelized if resources are available.
        :param interaction_times: list of initial arrival times for pulses and last entry is the arrival time for local
        oscillator.
        :param diagram: a double-sided diagram (ufss diagramGenerator format)
        :param t_range: a list of scan range for second pulse arrival time and detection time.
        :param r: time resolution (steps per fs)
        :param parallel: Parallelization control, True or False
        :return: a list of density matrices and a list of times
        '''

        if parallel:
            from qutip import parallel as pp

        coh1_time = np.linspace(0, t_range[0], t_range[0]*r)  # time list for the first delay
        rho = self.rho
        interaction = diagram[0]  # first pulse interaction
        print('appling first pulse at t1=0')
        rho = self.apply_pulse(rho, interaction)
        print('evolve state after first pulse ')
        results = mesolve(self.H, rho, coh1_time, self.c_ops, [])
        states1 = results.states  # collection of all states before second interaction
        #print('states after first coherence ', states1[-1])
        interaction = diagram[1]
        pop_time = np.linspace(0, t_range[1], t_range[1]*r)
        #print(states1[-1])
        print('applying 2nd pulse and evolving the population for fixed time')
        self.tlist = pop_time
        states1 = pp.parfor(self.para_mesolve2, states1, x=interaction, only_last_state=True)
        #print(states1[-1])
        #print('states after pop ', states1[-1])
        # at this point we have a whole list of states, each evolved for population time. These states will act as
        # initial states for next set of simulations.
        print('applying 3rd pulse and evolving afterwards')
        x = diagram[2]
        coh2_time = np.linspace(0, t_range[1], t_range[1]*r)
        final_states = []

        self.tlist = coh2_time
        final_states.append(pp.parfor(self.para_mesolve2, states1, x=x, only_last_state=False))
        return final_states[0]

    def coherence2D_pop(self, diagram, tau1, tau2, tau3, r=100, r2=7, parallel=False):
      """
      computes the third order response for a single diagram with fixed population time
      It can parallelized if resources are available
      """
      if parallel:
        from qutip import parallel as pp
      t1=0
      coh1_time = np.linspace(0,tau1,tau1*r)
      rho = self.rho
      x = diagram[0] # first pulse interaction
      print('appling first pulse at t1=0')
      rho = self.apply_pulse(rho, x)
      print('evolve state after first pulse ')
      results = mesolve(self.H, rho, coh1_time, self.c_ops, [])
      states1 = results.states #collection of all states before second interaction
      #print('states1 ', np.shape(states1))
      #print('states1 len',len(states1))
      #print('states after first coherence ', states1[-1])
      #########################################################################
      x = diagram[1] # select next pulse interaction
      pop_time = np.linspace(0, tau2, r2)
      print('population time list ', pop_time)
      #print(states1[-1])
      print('applying 2nd pulse and evolving the population for fixed time')
      self.tlist = pop_time
      states2 = pp.parfor(self.para_mesolve2, states1, x=x, only_last_state=False)
      states2 = np.swapaxes(states2, 0,1)
      #print(np.shape(states2))
      #print(len(states2))
      #print(states1[-1])
      #print('states after pop ', states1[-1])
      #at this point we have a whole list of states, which will act as initial state for final evolution
      print('applying 3rd pulse and evolving afterwards')
      x = diagram[2]
      coh2_time = np.linspace(0,tau3, tau3*r)
      final_states_all_pop = []
      self.tlist = coh2_time
      for j in range(len(pop_time)):
        states = states2[j]
        final_states = []
        final_states.append(pp.parfor(self.para_mesolve2, states1, x=x, only_last_state=False))
        final_states_all_pop.append(final_states[0])
      return final_states_all_pop

In [ ]:
    def multi_dimensional_coherence(self, time_list=None, diagram=None, r=10, parallel=False):
        '''
        computes the 2D coherence plot for a single diagram. First interaction is always at t=0.
        It can parallelized if resources are available.

        :param time_list: list of tuples. In each tuple, the first element is the time delay and the second element is a
        True/False statement indicating whether to scan the time delay.
        :param diagram: a double-sided diagram (ufss diagramGenerator format)
        :param r: time resolution (steps per fs)
        :param parallel: Parallelization control, True or False
        :return: a list of density matrices and a list of times
        '''

        if len(time_list) != len(diagram):
            print('pulse number and time delays do not match')
            return 0

        if parallel:
            from qutip import parallel as pp

        rho = self.rho  # taking the initial density matrix from the system class.
        for i in range(len(time_list)):
            t = time_list[i]
            interaction = diagram[i]
            if t[1]:
                t_scan = np.linspace(0, t[0], t[0]*r)  # creating a time scan if the time delay has to be scanned.
                results = mesolve(self.H, rho, t_scan, self.c_ops, [])
                states.append = results.states


In [ ]:
    def para_mesolve2(self, rho, tlist=None, x=None, only_last_state=True):
        rho = self.apply_pulse(rho, x)
        if only_last_state:
            return mesolve(self.H, rho, self.tlist, self.c_ops, []).states[-1]
        else:
            return mesolve(self.H, rho, self.tlist, self.c_ops, []).states

In [ ]:
    def multi_dimensional_coherence(self, time_list=None, diagram=None, r=10, parallel=False):
        '''
        computes the 2D coherence plot for a single diagram. First interaction is always at t=0.
        It can parallelized if resources are available.

        :param time_list: list of tuples. In each tuple, the first element is the time delay and the second element is a
        True/False statement indicating whether to scan the time delay.
        :param diagram: a double-sided diagram (ufss diagramGenerator format)
        :param r: time resolution (steps per fs)
        :param parallel: Parallelization control, True or False
        :return: a list of density matrices and a list of times
        '''

        if len(time_list) != len(diagram):
            print('pulse number and time delays do not match')
            return 0

        if parallel:
            from qutip import parallel as pp

        rho = self.rho  # taking the initial density matrix from the system class.
        for i in range(len(time_list)):
            t = time_list[i]
            interaction = diagram[i]
            if t[1]:
                t_scan = np.linspace(0, t[0], t[0]*r)  # creating a time scan if the time delay has to be scanned.
                results = mesolve(self.H, rho, t_scan, self.c_ops, [])
                states.append = results.states


In [ ]:
    def para_mesolve2(self, rho, tlist=None, x=None, only_last_state=True):
        rho = self.apply_pulse(rho, x)
        if only_last_state:
            return mesolve(self.H, rho, self.tlist, self.c_ops, []).states[-1]
        else:
            return mesolve(self.H, rho, self.tlist, self.c_ops, []).states